In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
df = pd.read_csv(r'C:\Users\vchan\.cache\kagglehub\datasets\nayakganesh007\google-ads-sales-dataset\versions\1\GoogleAds_DataAnalytics_Sales_Uncleaned.csv')
df

,Ad_ID,Campaign_Name,Clicks,Impressions,Cost,Leads,Conversions,Conversion Rate,Sale_Amount,Ad_Date,Location,Device,Keyword
0,A1000,DataAnalyticsCourse,104.0,4498.0,$231.88,14.0,7.0,0.058,$1892,2024-11-16,hyderabad,desktop,learn data analytics
1,A1001,DataAnalyticsCourse,173.0,5107.0,$216.84,10.0,8.0,0.046,$1679,20-11-2024,hyderabad,mobile,data analytics course
2,A1002,Data Anlytics Corse,90.0,4544.0,$203.66,26.0,9.0,NaN,$1624,2024/11/16,hyderabad,Desktop,data analitics online
3,A1003,Data Analytcis Course,142.0,3185.0,$237.66,17.0,6.0,NaN,$1225,2024-11-26,HYDERABAD,tablet,data anaytics training
4,A1004,Data Analytics Corse,156.0,3361.0,$195.9,30.0,8.0,NaN,$1091,2024-11-22,hyderabad,desktop,online data analytic
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2595,A3595,DataAnalyticsCourse,88.0,5344.0,$242.07,17.0,9.0,0.054,$1418,29-11-2024,HYDERABAD,MOBILE,online data analytic
2596,A3596,DataAnalyticsCourse,154.0,3211.0,$248.28,14.0,6.0,0.039,$1950,2024/11/28,hyderabad,TABLET,data analitics online
2597,A3597,Data Anlytics Corse,113.0,3808.0,$233.25,18.0,4.0,0.035,$1085,2024-11-02,Hyderbad,desktop,data anaytics training
2598,A3598,Data Analytics Corse,196.0,5853.0,$220.13,16.0,7.0,0.036,$1558,2024-11-08,hydrebad,Tablet,data anaytics training


In [3]:
df.isnull().sum()

Ad_ID                0
Campaign_Name        0
Clicks             112
Impressions         54
Cost                97
Leads               48
Conversions         74
Conversion Rate    626
Sale_Amount        139
Ad_Date              0
Location             0
Device               0
Keyword              0
dtype: int64

In [4]:
df['Campaign_Name'] = df['Campaign_Name'].replace({'DataAnalyticsCourse':'Data Analytics Course','Data Anlytics Corse':'Data Analytics Course','Data Analytics Corse':'Data Analytics Course','Data Analytcis Course':'Data Analytics Course'})

In [5]:
df['Location'] = df['Location'].replace({'hyderabad':'Hyderabad','HYDERABAD':'Hyderabad','Hyderbad':'Hyderabad','hydrebad':'Hyderabad'})

In [6]:
df['Device'] = df['Device'].replace({
    'desktop': 'Desktop', 'Desktop': 'Desktop', 'DESKTOP': 'Desktop',
    'mobile': 'Mobile',   'Mobile': 'Mobile',   'MOBILE': 'Mobile',
    'tablet': 'Tablet',   'Tablet': 'Tablet',   'TABLET': 'Tablet'
})


In [7]:
df['Impressions'] = df['Impressions'].fillna(df['Impressions'].median())

In [11]:
df['Clicks'] = df.groupby(
    ['Campaign_Name','Location','Device']
)['Clicks'].transform(lambda x:x.fillna(x.median()))

In [16]:
df['Cost'] = (df['Cost'].astype(str).str.replace('$','',regex=False).astype(float))

In [17]:
df['CPC'] = df['Cost']/df['Clicks']

In [19]:
df['Cost'] = df['Cost'].fillna(
    df['Clicks'] *
    df.groupby(['Campaign_Name','Device'])['CPC'].transform('median')
)

In [23]:
lead_calculation = (df['Leads'] / df['Clicks']).median()
df['Leads'] = df['Leads'].fillna(df['Clicks'] * lead_calculation).round()

In [27]:
conversion_rate = (df['Conversions'] / df['Leads']).median()
df['Conversions'] = df['Conversions'].fillna(df['Leads'] * conversion_rate).round()

## Here how many people got converted by ads is important since it is google ad data so thats why we are using clicks

In [33]:
df['Conversion Rate'] = df['Conversions'] / df['Clicks']


In [42]:
df['Sale_Amount'] = df['Sale_Amount'].astype(str).str.replace('$','',regex=False).astype(float)

In [43]:
sales_amt = (df['Sale_Amount'].sum() / df['Conversions'].sum()).round()

In [44]:
df['Sale_Amount'] = df['Sale_Amount'].fillna(df['Conversions'] * sales_amt)

In [47]:
df['CPC'] = df['Cost'] / df['Clicks']

In [50]:
df['Keyword'] = (
    df['Keyword']
    .str.lower()
    .str.strip()
)


In [51]:
def keyword_category(k):
    if 'course' in k or 'traning' in k:
        return 'course_intent'
    elif 'learn' in k:
        return 'learning_intent'
    elif 'online' in k:
        return 'online_intent'
    else:
        return 'general_intent'

df['Keyword_Category'] = df['Keyword'].apply(keyword_category)


In [70]:
df

,Ad_ID,Campaign_Name,Clicks,Impressions,Cost,Leads,Conversions,Conversion Rate,Sale_Amount,Ad_Date,Location,Device,Keyword,CPC,Keyword_Category
0,A1000,Data Analytics Course,104.0,4498.0,231.880000,14.0,7.0,0.067308,1892.0,2024-11-16,Hyderabad,Desktop,learn data analytics,2.229615,learning_intent
1,A1001,Data Analytics Course,173.0,5107.0,216.840000,10.0,8.0,0.046243,1679.0,2024-11-16,Hyderabad,Mobile,data analytics course,1.253410,course_intent
2,A1002,Data Analytics Course,90.0,4544.0,203.660000,26.0,9.0,0.100000,1624.0,2024-11-16,Hyderabad,Desktop,data analitics online,2.262889,online_intent
3,A1003,Data Analytics Course,142.0,3185.0,237.660000,17.0,6.0,0.042254,1225.0,2024-11-26,Hyderabad,Tablet,data anaytics training,1.673662,general_intent
4,A1004,Data Analytics Course,156.0,3361.0,195.900000,30.0,8.0,0.051282,1091.0,2024-11-22,Hyderabad,Desktop,online data analytic,1.255769,online_intent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2595,A3595,Data Analytics Course,88.0,5344.0,242.070000,17.0,9.0,0.102273,1418.0,2024-11-16,Hyderabad,Mobile,online data analytic,2.750795,online_intent
2596,A3596,Data Analytics Course,154.0,3211.0,248.280000,14.0,6.0,0.038961,1950.0,2024-11-16,Hyderabad,Tablet,data analitics online,1.612208,online_intent
2597,A3597,Data Analytics Course,113.0,3808.0,233.250000,18.0,4.0,0.035398,1085.0,2024-11-02,Hyderabad,Desktop,data anaytics training,2.064159,general_intent
2598,A3598,Data Analytics Course,196.0,5853.0,220.130000,16.0,7.0,0.035714,1558.0,2024-11-08,Hyderabad,Tablet,data anaytics training,1.123112,general_intent


In [ ]:
df.to_csv('Google_Ads.csv',index=False)